In [1]:
import cobra.test
import cobra
from cobra import Model, Reaction, Metabolite
import os
import pandas as pd
from Bio import SeqIO
import pickle
import numpy as np

In [2]:
HP_model = cobra.io.read_sbml_model("draft_model/HP_draft_model.xml")

<h3> Add missing ATP reaction </h3>

In [3]:
model_coeli = pickle.load(open('template_model/sco_model.p','rb'))
'''
summary = model_coeli.metabolites.atp_c.summary()
summary = summary.to_frame()
rxns = summary.index.get_level_values(1).to_list()

gene_ids = []

for rxn in rxns:
    print("Genes")
    print("-----")
    for x in model_coeli.genes:
        associated_ids = [i.id for i in x.reactions if i.id == rxn]
        if len(associated_ids)>0:
            gene_ids.append([x.id, associated_ids[0]])
            print("%s is associated with reactions: %s" %
                  (x.id, associated_ids))
    print("")

print(gene_ids)
'''

'\nsummary = model_coeli.metabolites.atp_c.summary()\nsummary = summary.to_frame()\nrxns = summary.index.get_level_values(1).to_list()\n\ngene_ids = []\n\nfor rxn in rxns:\n    print("Genes")\n    print("-----")\n    for x in model_coeli.genes:\n        associated_ids = [i.id for i in x.reactions if i.id == rxn]\n        if len(associated_ids)>0:\n            gene_ids.append([x.id, associated_ids[0]])\n            print("%s is associated with reactions: %s" %\n                  (x.id, associated_ids))\n    print("")\n\nprint(gene_ids)\n'

In [4]:
reaction_atps4r = model_coeli.reactions.ATPS4r
HP_model.add_reactions([reaction_atps4r])

#check corresponding sco genes from template_model/temp_target_BBH_dict.txt

HP_model.reactions.ATPS4r.gene_reaction_rule = 'NJMEEBPL_03421 and NJMEEBPL_03423 and NJMEEBPL_03416 and NJMEEBPL_03417 and NJMEEBPL_03418 and NJMEEBPL_03419 and NJMEEBPL_03420'

In [6]:
reaction_ACCOAC = model_coeli.reactions.ACCOAC
HP_model.add_reactions([reaction_ACCOAC])
HP_model.reactions.ACCOAC.gene_reaction_rule = '( NJMEEBPL_07611 or ( NJMEEBPL_03221 and NJMEEBPL_03220 ) ) and ( NJMEEBPL_06272 or NJMEEBPL_05660 or NJMEEBPL_09895 )'

In [7]:
solution = HP_model.optimize()
print(solution)
print(solution.objective_value)

<Solution 0.074 at 0x7f49225be610>
0.07407495361456679


In [8]:
HP_model.summary()

In [9]:
HP_model.metabolites.atp_c.summary()

<h3> Add rapamycin producing pathway </h3>

In [10]:
def add_metabolites(io, sheet_name, model):
    df_metab = pd.read_excel(io, sheet_name)
    for i in range(df_metab.shape[0]):
        metab = Metabolite(df_metab.ID[i], 
                           formula = df_metab.Formula[i],
                           name = df_metab.Name[i],
                           compartment = df_metab.Compartment[i])
        model.add_metabolites([metab])
        print(metab.id,' is added to the model.')

In [11]:
def add_reactions(io, sheet_name, model):
    df_reac = pd.read_excel(io, sheet_name)
    for i in range(df_reac.shape[0]):
        reac = Reaction(df_reac.ID[i])
        reac.name = df_reac.Name[i]
        model.add_reactions([reac])
        reac.build_reaction_from_string(df_reac.Equation[i], fwd_arrow = '=>', reversible_arrow = '<=>')
        
        if df_reac.Reversible[i] == 'Y':
            reac.lower_bound = -1000.0
            reac.upper_bound = 1000.0
        else:
            reac.lower_bound = 0.0
            reac.upper_bound = 1000.0
        
        
        print(reac.id, ' is added to the model')

In [12]:
add_metabolites('inputs/metabolite_reaction.xlsx', 0, HP_model)

dhchc_c  is added to the model.
pipecol__L_c  is added to the model.
rap_c  is added to the model.


In [13]:
add_reactions('inputs/metabolite_reaction.xlsx', 1, HP_model)

DHCHCS  is added to the model
MNXR113634  is added to the model
Rapamycin  is added to the model


In [14]:
HP_model.reactions.EX_glc__D_e.lower_bound = -10.0
#HP_model.reactions.EX_glyc_e.lower_bound = -5.0
#HP_model.reactions.BIOMASS_SCO.lower_bound = 0.75*1.309
DM_rap = HP_model.add_boundary(HP_model.metabolites.rap_c, type='demand')
HP_model.objective = HP_model.reactions.Rapamycin
HP_model.optimize()

,fluxes,reduced_costs
ACTS10,0.000000,-2.170680e-03
ACTS13,0.000000,-2.170680e-03
ACTS14,0.000000,0.000000e+00
ACTS15,0.000000,0.000000e+00
ACTS16,0.000000,0.000000e+00
...,...,...
ACCOAC,0.995594,0.000000e+00
DHCHCS,0.142228,-4.857226e-17
MNXR113634,0.142228,-2.428613e-16
Rapamycin,0.142228,1.172673e-15


In [15]:
cobra.io.write_sbml_model(HP_model,"final_model/HP_model_final.xml")